# SAMOS Spectroscopy - Trace Extraction and Rectification

## Status: ⚠️ In Development

This notebook is being updated to use the new `samos` package structure.

**What this notebook does:**
1. Fits polynomial to trace curvature
2. Performs flat fielding
3. Subtracts background/sky
4. Rectifies curved spectra to straight traces
5. Extracts 1D spectra

**Prerequisites:**
- Run Notebook 01 first (creates `spec_*.fits` files)

---

## Current Workaround

**Option 1: Use the old notebook**
- The original algorithms are preserved in `04_trace_extraction_OLD.ipynb`
- They still work, just use old `Class_SAMOS` structure
- You can adapt the code manually if needed

**Option 2: Use external tools**
- **PypeIt**: Professional spectroscopic reduction
  - https://pypeit.readthedocs.io/
  - Handles trace fitting, extraction, everything
  - Install: `pip install pypeit`

**Option 3: Wait for full update**
- This notebook will be fully updated soon
- Check back or request priority

---

## What Needs to Be Done

To complete this notebook, we need to create:

1. **`samos/spectroscopy/extraction.py`** module with:
   - Trace fitting functions (polynomial, spline)
   - Background subtraction
   - Flat fielding
   - Rectification
   - 1D extraction (boxcar and optimal)

2. **Updated notebook** that uses the new module

---

## Simplified Workflow (What You Can Do Now)

Here's a simplified version of the key steps using available tools:

In [ ]:
# Standard imports
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.io import fits
import cv2
from scipy import ndimage, signal

# SAMOS package
from samos.utils import io, display

print("✓ Packages loaded")

## Configuration

In [ ]:
# Target information
target_name = "ABELL3120"
target_mode = "SAMI_manual_Mask_T00_Low_Red"

# Working directory (where spec_*.fits files are from notebook 01)
analysis_top_directory = "/Users/nestrada/Documents/SAMOS/SAMOS_REDUCED"
working_directory = os.path.join(analysis_top_directory, target_name, target_mode)

os.chdir(working_directory)
print(f"Working directory: {working_directory}")

# Find all spec files
spec_files = sorted(Path('.').glob('spec_*.fits'))
print(f"Found {len(spec_files)} slit files to process")

## Important Note

The cells below show the **concept** of what needs to be done for each step.

For a **complete, production-ready implementation**, either:
1. Use the algorithms from `04_trace_extraction_OLD.ipynb`
2. Use PypeIt (recommended for publication-quality results)
3. Wait for the full `samos.spectroscopy.extraction` module

---

## Step 1: Load Slit Data

In [ ]:
# Example: Load first slit
i_slit = 0
fits_file = f'spec_{i_slit:03d}.fits'

print(f"Loading: {fits_file}")
hdul = fits.open(fits_file)
hdul.info()

# Extract data
header = hdul[0].header
data = hdul[1].data        # Science data
flat = hdul[2].data        # Flat field
lines = hdul[3].data       # Arc lamp

# Get slit boundaries from header
slits = [header['Y0'], header['Y1'], header['Y2'], header['Y3']]
print(f"Slit {i_slit}: y=[{slits[0]}:{slits[3]}]")
print(f"Data shape: {data.shape}")

# Display
vmin, vmax = display.get_percentile_limits(data, 1, 99)
display.display_image(data, zmin=vmin, zmax=vmax, 
                     title=f'Slit {i_slit} - Raw 2D Spectrum')

hdul.close()

## Step 2: Fit Trace Curvature

This finds how the spectral trace curves across the detector.

In [ ]:
# Simple edge detection function
def find_edges(data, threshold_fraction=0.2):
    """Find edges in 1D profile."""
    threshold = (max(data) - min(data)) * threshold_fraction
    gradient = [data[i] - data[i-1] for i in range(1, len(data))]
    edges = [i for i, grad in enumerate(gradient) if abs(grad) > threshold]
    return edges

# Find trace center at each x position
dx, dy = flat.shape[1], flat.shape[0]
xpix, yctr = [], []

for ix in range(dx):
    profile = flat[:, ix]
    edges = find_edges(profile)
    
    if len(edges) >= 2:
        e0, e1 = edges[0], edges[-1]
        xpix.append(ix)
        yctr.append(np.mean([e0, e1]))

# Find stable region (where trace doesn't vary much)
x_good = []
for ix in range(5, len(yctr) - 50):
    if np.std(yctr[ix+1:ix+50]) < 1:  # Stable region
        x_good.append(ix)

if len(x_good) > 100:
    imin, imax = min(x_good), max(x_good)
    
    # Fit 3rd degree polynomial
    coeffs = np.polyfit(xpix[imin:imax], yctr[imin:imax], 3)
    trace_poly = np.poly1d(coeffs)
    
    # Visualize fit
    plt.figure(figsize=(12, 4))
    plt.ylim(10, 30)
    plt.scatter(xpix[imin:imax], yctr[imin:imax], s=1, alpha=0.5, label='Detected edges')
    plt.plot(xpix, trace_poly(xpix), 'r-', linewidth=2, label='Polynomial fit')
    plt.xlabel('X pixel (dispersion)')
    plt.ylabel('Y pixel (spatial)')
    plt.title('Trace Curvature')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()
    
    print(f"✓ Trace fit successful (x range: {imin}-{imax})")
    print(f"Polynomial coefficients: {coeffs}")
else:
    print("⚠️ Could not find stable trace region")
    trace_poly = None

## Step 3: Create Masks and Flat Field

Separate the illuminated slit from sky regions.

In [ ]:
if trace_poly is not None:
    # Create binary masks
    mask_in = np.zeros_like(flat)
    mask_out = np.zeros_like(flat)
    
    # Define slit edges relative to trace
    below = slits[1] - slits[0] - 2
    above = slits[3] - slits[2] - 2
    
    # Fill masks column by column
    for ix in range(dx):
        trace_y = int(trace_poly(ix))
        mask_out[0:max(0, trace_y - below), ix] = 1
        mask_out[min(dy, trace_y + above):, ix] = 1
    
    mask_in = 1 - mask_out
    
    # Display masks
    display.display_comparison(
        [mask_in, mask_out],
        ['Slit Mask (illuminated)', 'Sky Mask (background)'],
        zmin=0, zmax=1
    )
    
    print("✓ Masks created")
else:
    print("⚠️ Skipping mask creation (no trace fit)")

## Remaining Steps (Concept)

The following steps are **conceptual outlines**. For full implementation, see:
- `04_trace_extraction_OLD.ipynb` - Original working algorithms
- PypeIt documentation - Professional implementation

### Step 4: Background Subtraction
```python
# Extract background from masked regions
# Subtract column-by-column
# Handle OH sky lines
```

### Step 5: Flat Fielding
```python
# Normalize flat field
# Smooth along dispersion direction
# Divide science data by flat
```

### Step 6: Rectification
```python
# Resample 2D spectrum
# Straighten curved traces
# Use trace polynomial to shift each column
```

### Step 7: 1D Extraction
```python
# Sum rows within trace (boxcar)
# Or: Optimal extraction (weighted by profile)
# Save as spec1d_*.fits
```

---

## Using PypeIt (Recommended)

For publication-quality results, consider using PypeIt:

```python
# Install PypeIt
# pip install pypeit

from pypeit import extraction
from pypeit import flatfield
from pypeit import tracepca

# PypeIt has robust algorithms for all these steps
# See: https://pypeit.readthedocs.io/
```

---

## Full Algorithm Reference

The complete, tested algorithms are in `04_trace_extraction_OLD.ipynb`.

Key differences from this simplified version:
- More robust edge detection
- Gaussian smoothing of flat field
- Sophisticated sky subtraction (median of lowest pixels)
- High-order resampling for rectification
- Proper handling of NaN values

To use the old algorithms:
1. Open `04_trace_extraction_OLD.ipynb`
2. Update the imports (remove `Class_SAMOS`)
3. Use `samos` package functions where appropriate
4. The core algorithms will still work!

---

## Status and Next Steps

**Current status**: Simplified workflow provided above

**For full functionality**:
1. Use `04_trace_extraction_OLD.ipynb` algorithms
2. Or wait for `samos.spectroscopy.extraction` module
3. Or use PypeIt for professional results

**To request full update**:
- Contact development team
- Specify if this is high priority for your work
- Share example data for testing